In [ ]:
!python --version

In [ ]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
from zipfile import ZipFile
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
import random
from glob import glob
import seaborn as sns
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import matplotlib.image as img
import warnings
import shutil
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth = True
except Exception as ex:
    print(e)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
with ZipFile("/content/drive/MyDrive/archive.zip", "r") as f:
    f.extractall(path="/content/images/")

In [ ]:
os.mkdir("/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/not-melanoma/")

for _, dirs, _ in os.walk("/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/"):
    if len(dirs) != 0:
        for d in dirs:
            if d == "melanoma" or d == "not-melanoma":
                continue
            for _, _, files in os.walk(f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/{d}"):
                for f in files:
                    shutil.move(f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/{d}/{f}", f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/not-melanoma/{f}")
            shutil.rmtree(f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/{d}")



In [ ]:
os.mkdir("/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/not-melanoma/")

for _, dirs, _ in os.walk("/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/"):
    if len(dirs) != 0:
        for d in dirs:
            if d == "melanoma" or d == "not-melanoma":
                continue
            for _, _, files in os.walk(f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/{d}"):
                for f in files:
                    shutil.move(f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/{d}/{f}", f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/not-melanoma/{f}")
            shutil.rmtree(f"/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/{d}")



In [ ]:
data_dir_train = pathlib.Path("images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/")
data_dir_test = pathlib.Path("images/Skin cancer ISIC The International Skin Imaging Collaboration/Test/")

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
random.seed(2137)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="training",
  seed=2137,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_train,
  validation_split=0.2,
  subset="validation",
  seed=1327,
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir_test,
  validation_split=0.9,
  subset="validation",
  seed=3721,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
!pip install Augmentor

In [ ]:
path_to_training_dataset = '/content/images/Skin cancer ISIC The International Skin Imaging Collaboration/Train/'
import Augmentor
for c in class_names:
    pipeline = Augmentor.Pipeline(path_to_training_dataset + c, output_directory=f"/content/balanced/{c}/")
    pipeline.rotate(probability=0.7, max_left_rotation=20, max_right_rotation=20)
    pipeline.zoom(probability=0.4, min_factor=1.1, max_factor=1.6)
    pipeline.flip_left_right(probability=0.5)
    pipeline.flip_top_bottom(probability=0.5)
    pipeline.sample(1800)

In [ ]:
output_dir = pathlib.Path('/content/balanced/')
image_count_train = len(list(output_dir.glob('*/*.jpg')))
print(image_count_train)

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  output_dir,
  seed=2137,
  validation_split = 0.2,
  subset = 'training',
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  output_dir,
  seed=3127,
  validation_split = 0.2,
  subset = 'validation',
  image_size=(img_height, img_width),
  batch_size=batch_size)


In [ ]:
print(train_ds.class_names)

In [ ]:
num_classes = 2
model = tf.keras.applications.densenet.DenseNet121(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    classifier_activation='softmax'
)

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(optimizer= opt,
              loss = SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
epochs = 25
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save("model_full_densenet_isic18_97_88.h5")

In [ ]:
eval_loss, eval_accuracy = model.evaluate(test_ds, batch_size=batch_size, verbose=1)